Данный ноутбук - выжимка идей, которые использовались в решении

Для подсчета нужно 16Гб оперативы, иначе признаки на precip (осадки) не получится подсчитать.

Нужен набор https://www.esrl.noaa.gov/psd/thredds/catalog/Datasets/cpc_global_precip/catalog.html

In [19]:
import os
import random
import gzip
from sklearn.linear_model import LinearRegression
import catboost
from catboost import EFstrType
import numpy as np
import pandas as pd
import xarray
from sklearn.metrics import roc_auc_score
import scipy as sp
import scipy.fftpack
import gc
import geopandas
from catboost import CatBoostRegressor
import gdal, ogr
from shapely import wkb
import pickle
from shapely.geometry import box
from collections import defaultdict
from tqdm import tqdm
from datetime import date, datetime
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.mixture import GaussianMixture
import math
SEED = 42
SEED2 = 70
VAL_MONTHS = 6

ITERATIONS = 2100

DATA_PATH = '../data'
MODELS_PATH = './cb'
OSM_GEO_DATA = os.path.join(MODELS_PATH, 'russia.osm.gpkg.gz')

pd.set_option("max_columns", 500)

In [2]:
def reseed(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)


def evaluate(y_true, y_pred, high_level=12):
    gt = np.zeros_like(y_pred, dtype=np.int8)
    gt[np.arange(y_true.shape[0]), y_true - 1] = 1
    result = {'roc_auc_micro': roc_auc_score(gt, y_pred, average='micro')}
    for ft in range(1, high_level):
        gt = (y_true == ft)
        if gt.max() == gt.min():
            roc_auc = 0
        else:
            roc_auc = roc_auc_score(gt, y_pred[:, ft - 1])
        result[f'roc_auc_{ft}'] = roc_auc
    return result

In [3]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)

seasons = [('1', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('2', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('3', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('4', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('1', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

def preprocess(df):
    df['longitude'] = df['longitude'].astype(np.float32)
    df['latitude'] = df['latitude'].astype(np.float32)
    df['weekday'] = df.date.dt.weekday.astype(np.int8)
    df['year'] = df['date'].dt.year
    df['day'] = df['date'].dt.day
    df['weekends'] = df['weekday'] < 5
    df['weekends'] = df['weekends'].astype(np.int8)
    df['passed_years'] = date.today().year - df['date'].dt.year
    df['passed_months'] = (date.today().year - df['date'].dt.year) * 12 + date.today().month - df['date'].dt.month
    df['seasons'] = df['date'].apply(lambda x: get_season(x))
    df['month'] = df.date.dt.month.astype(np.int8)
    
    df['ym'] = (df.date.dt.month + (df.date.dt.year - 2000) * 12).astype(np.int16)
    df['fire_type'] = df.fire_type.astype(np.uint8)
    df.set_index('fire_id', inplace=True)
    df.drop(['fire_type_name'], axis=1, inplace=True)


In [4]:
def day_length(latitude, month):

    day_length46N = [6.5, 7.5, 9.0, 12.8, 13.9, 13.9, 12.4, 10.9, 9.4, 8.0, 7.0, 6.0]
    day_length20N = [7.9, 8.4, 8.9, 9.5, 9.9, 10.2, 10.1, 9.7, 9.1, 8.6, 8.1, 7.8]
    day_length20S = [10.1, 9.6, 9.1, 8.5, 8.1, 7.8, 7.9, 8.3, 8.9, 9.4, 9.9, 10.2]
    day_length40S = [11.5, 10.5, 9.2, 7.9, 6.8, 6.2, 6.5, 7.4, 8.7, 10.0, 11.2, 11.8]

    ret_val = None

    if latitude <= 90 and latitude > 33:
        ret_val = day_length46N[month - 1]
    elif latitude <= 33 and latitude > 0.0:
        ret_val = day_length20N[month - 1]
    elif latitude <= 0.0 and latitude > -30.0:
        ret_val = day_length20S[month - 1]
    elif latitude <= -30.0 and latitude >= -90.0:
        ret_val = day_length40S[month - 1]

    if ret_val == None:
        ret_val = 9.18

    return ret_val


def ffmc(temp, rh, wind, rain, ffmc_prev=85):
    rh = min(100.0, rh)
    mo = 147.2 * (101.0 - ffmc_prev) / (59.5 + ffmc_prev)

    if rain > .5:
        rf = rain - .5

        if mo <= 150.0:
            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * (1.0 - math.exp(-6.93 / rf))
        else:

            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * (1.0 - math.exp(-6.93 / rf)) + \
                 0.0015 * pow(mo - 150.0, 2) * pow(rf, .5)

        if mr > 250.0:
            mr = 250.0

        mo = mr

    ed = 0.942 * pow(rh, 0.679) + \
         11.0 * math.exp((rh - 100.0) / 10.0) + 0.18 * (21.1 - temp) * (1.0 - math.exp(-0.115 * rh))

    if mo > ed:
        ko = 0.424 * (1.0 - pow(rh / 100.0, 1.7)) + \
             0.0694 * pow(wind, .5) * (1.0 - pow(rh / 100.0, 8))

        kd = ko * 0.581 * math.exp(0.0365 * temp)

        m = ed + (mo - ed) * pow(10.0, -kd)

    else:
        ew = 0.618 * pow(rh, 0.753) + \
             10.0 * math.exp((rh - 100.0) / 10.0) + \
             0.18 * (21.1 - temp) * (1.0 - math.exp(-0.115 * rh))
        if mo < ew:
            k1 = 0.424 * (1.0 - pow((100.0 - rh) / 100.0, 1.7)) + \
                 0.0694 * pow(wind, .5) * (1.0 - pow((100.0 - rh) / 100.0, 8))

            kw = k1 * 0.581 * math.exp(0.0365 * temp)

            m = ew - (ew - mo) * pow(10.0, -kw)
        else:
            m = mo
    result = 59.5 * (250.0 - m) / (147.2 + m)
    if type(result) == complex:
        return result.real
    return result


def dmc(temp, rh, rain, dmc_prev, lat, month):
    rh = min(100.0, rh)
    if rain > 1.5:
        re = 0.92 * rain - 1.27

        mo = 20.0 + math.exp(5.6348 - dmc_prev / 43.43)

        if dmc_prev <= 33.0:
            b = 100.0 / (0.5 + 0.3 * dmc_prev)
        else:
            if dmc_prev <= 65.0:
                b = 14.0 - 1.3 * math.log(dmc_prev)
            else:
                b = 6.2 * math.log(dmc_prev) - 17.2

        mr = mo + 1000.0 * re / (48.77 + b * re)

        pr = 244.72 - 43.43 * math.log(mr - 20.0)

        if pr > 0.0:
            dmc_prev = pr
        else:
            dmc_prev = 0.0

    if temp > -1.1:
        d1 = day_length(lat, month)

        k = 1.894 * (temp + 1.1) * (100.0 - rh) * d1 * 0.000001

    else:
        k = 0.0

    result = dmc_prev + 100.0 * k
    if type(result) == complex:
        return result.real
    return result


def dc(temp, rain, dc_prev, lat, month):

    if rain > 2.8:
        rd = 0.83 * rain - 1.27
        Qo = 800.0 * math.exp(-dc_prev / 400.0)
        Qr = Qo + 3.937 * rd
        Dr = 400.0 * math.log(800.0 / Qr)

        if Dr > 0.0:
            dc_prev = Dr
        else:
            dc_prev = 0.0

    Lf = drying_factor(lat, month - 1)

    if temp > -2.8:
        V = 0.36 * (temp + 2.8) + Lf
    else:
        V = Lf

    if V < 0.0:
        V = 0.0

    D = dc_prev + 0.5 * V

    result = D
    if type(result) == complex:
        return result.real
    return result


def isi(wind, ffmc):

    fwind = math.exp(0.05039 * wind)

    m = 147.2 * (101.0 - ffmc) / (59.5 + ffmc)

    fF = 91.9 * math.exp(-0.1386 * m) * (1.0 + pow(m, 5.31) / 49300000.0)

    result = 0.208 * fwind * fF
    if type(result) == complex:
        return result.real
    return result


def bui(dmc, dc):

    if dmc <= 0.4 * dc:
        U = 0.8 * dmc * dc / (dmc + 0.4 * dc)
    else:
        U = dmc - (1.0 - 0.8 * dc / (dmc + 0.4 * dc)) * \
            (0.92 + pow(0.0114 * dmc, 1.7))

    result = max(U, 0.0)
    if type(result) == complex:
        return result.real
    return result


def fwi(isi, bui):
    if bui <= 80.0:
        fD = 0.626 * pow(bui, 0.809) + 2.0
    else:
        fD = 1000.0 / (25.0 + 108.64 * math.exp(-0.023 * bui))

    B = 0.1 * isi * fD

    if B > 1.0:
        S = math.exp(2.72 * pow(0.434 * math.log(B), 0.647))
    else:
        S = B

    result = S
    if type(result) == complex:
        return result.real
    return result


def drying_factor(latitude, month):
    LfN = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]
    LfS = [6.4, 5.0, 2.4, 0.4, -1.6, -1.6, -1.6, -1.6, -1.6, 0.9, 3.8, 5.8]

    if latitude > 0:
        ret_val = LfN[month]
    elif latitude <= 0.0:
        ret_val = LfS[month]

    return ret_val

In [26]:

class KFoldTargetEncoderTest(BaseEstimator, TransformerMixin):
    
    def __init__(self,train,colNames,encodedName):
        
        self.train = train
        self.colNames = colNames
        self.encodedName = encodedName
        
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        mean =  self.train[[self.colNames,
                self.encodedName]].groupby(
                                self.colNames).mean().reset_index() 
        
        dd = {}
        for index, row in mean.iterrows():
            dd[row[self.colNames]] = row[self.encodedName]
        X[self.encodedName] = X[self.colNames]
        X = X.replace({self.encodedName: dd})
        
        return X


class KFoldTargetEncoderTrain(BaseEstimator, TransformerMixin):
    def __init__(self,colnames,targetName,
                  n_fold=5, verbosity=True,
                  discardOriginal_col=False):
        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)
        mean_of_target = X[self.targetName].mean()
        kf = KFold(n_splits = self.n_fold,
                   shuffle = True, random_state=2019)
        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan
        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
            X.loc[X.index[val_ind], col_mean_name] =   \
            X_val[self.colnames].map(X_tr.groupby(self.colnames)
                                     [self.targetName].mean())
            X[col_mean_name].fillna(mean_of_target, inplace = True)
        if self.verbosity:
            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} \
                   is {}.'.format(col_mean_name,self.targetName, 
                   sp.stats.spearmanr(X[self.targetName].values,
                               encoded_feature)))
#                    np.corrcoef(X[self.targetName].values,
#                                encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
        return X


In [6]:
def load_ncep_var(var, press_level, suffix=''):
    result = []
    is_level = False
#     year = 2019
    for year in range(2012, 2020):
        dataset_filename = '{}/ncep/{}{}.{}.nc'.format(DATA_PATH, var, suffix, year)
#         dataset_filename = 'ncep/{}.{}.nc'.format(var, year)
        print("File name: ", dataset_filename)
        ds = xarray.open_dataset(dataset_filename)
        if 'level' in ds.coords:
            ds = ds.sel(drop=True, level=press_level)[var]
            is_level = True
        else:
            ds = ds.sel(drop=True)[var]
        ds = ds[:, (ds.lat >= 15 * 2.5 - 0.1) & (ds.lat <= 29 * 2.5 + 0.1),
             (ds.lon >= 6 * 2.5 - 0.1) & (ds.lon <= 71 * 2.5 + 0.1)]
        result.append(ds)
    ds = xarray.merge(result)
    df = ds.to_dataframe()[[var]].reset_index()

    df = df.merge(ds.rolling(time=3,center=True,min_periods=1).mean().to_dataframe()[[var]].reset_index(),
                  on=['lon', 'lat', 'time'], suffixes=('', '_3d'), how='left')
    gc.collect()

    if is_level:
        df = df.merge(ds.rolling(time=7,center=True,min_periods=1).mean().to_dataframe()[[var]].reset_index(),
                      on=['lon', 'lat', 'time'], suffixes=('', '_7d'), how='left')
        gc.collect()
        df = df.merge(ds.rolling(time=14,center=True,min_periods=1).mean().to_dataframe()[[var]].reset_index(),
                      on=['lon', 'lat', 'time'], suffixes=('', '_14d'), how='left')
        gc.collect()
        df = df.merge(ds.rolling(time=28,center=True,min_periods=1).mean().to_dataframe()[[var]].reset_index(),
                      on=['lon', 'lat', 'time'], suffixes=('', '_28d'), how='left')

        df['lat'] = np.round(df.lat / 2.5).astype(np.int8)
        df['lon'] = np.round(df.lon / 2.5).astype(np.int8)
    else:
        df['lon_int'] = df['lon'].astype(np.int16)
        df['lat_int'] = df['lat'].astype(np.int16)
    return df.copy()


def add_ncep_features(df):
    df['lon'] = np.round(df.longitude / 2.5).astype(np.int8)
    df['lat'] = np.round(df.latitude / 2.5).astype(np.int8)
    
    df['lon_int'] = df['longitude'].astype(np.int16)
    df['lat_int'] = df['latitude'].astype(np.int16)
    
    for var, press_level in (('air', 1000), ('uwnd', 1000), ('rhum', 1000)):
        var_df = load_ncep_var(var, press_level)
        mdf = df.reset_index().merge(var_df, left_on=['lon', 'lat', 'date'], right_on=['lon', 'lat', 'time'],
                                     how='left', ).set_index('fire_id')
        for suffix in ('', '_3d', '_7d', '_14d', '_28d'):
            df[var + suffix] = mdf[var + suffix].values
            
    for var in ['precip']:
        var_df = load_ncep_var(var, 1000)
        mdf = df.reset_index().merge(var_df, left_on=['lon_int', 'lat_int', 'date'], right_on=['lon_int', 'lat_int', 'time'],
                                     how='left').set_index('fire_id')
        for suffix in ('', '_3d'):
            df[var + suffix] = mdf.groupby(['longitude', 'latitude', 'date'])[var + suffix]\
                                                .mean().reset_index()[var + suffix].fillna(0)
    
    df['month_air_mean'] = df.groupby(["month"])['air'].transform("mean")
    df['month_air_std'] = df.groupby(["month"])['air'].transform("std")
    
    df['seasons_uwnd_mean'] = df.groupby(["seasons"])['uwnd'].transform("mean")
    df['seasons_uwnd_std'] = df.groupby(["seasons"])['uwnd'].transform("std")
    
    df['year_rhum_mean'] = df.groupby(["year"])['rhum'].transform("mean")
    df['year_rhum_std'] = df.groupby(["year"])['rhum'].transform("std")
    
    group_ym_rhum = df.groupby(["year", "month"])
    df['year_month_rhum_mean'] = group_ym_rhum['rhum'].transform("mean")
    df['year_month_rhum_std'] = group_ym_rhum['rhum'].transform("std")   
    
    df['month_rhum_min'] = df.groupby(["month"])['rhum'].transform("min")
    df['month_rhum_max'] = df.groupby(["month"])['rhum'].transform("max")
    
    df['x'] = np.cos(df.longitude)*np.cos(df.latitude)
    df['y'] = np.sin(df.latitude)*np.cos(df.longitude)
    df['z'] = np.sin(df.latitude)
    
    df['x_p'] = np.sin(df.latitude)
    df['z_p'] = np.cos(df.longitude)
    
    df['temp_psd'] = np.abs(sp.fftpack.fft(df['air']) ** 2)
    df['temp_fft_freq'] = sp.fftpack.fftfreq(len(df['temp_psd']), 1. / 365)
    df['temp_psd'] = np.log2(df['temp_psd']+1e-7)
       
    ffmc_prev = 85
    dmc_prev = 6
    dc_prev = 15
    ffmc = []
    dmc = []
    dc = []
    isi = []
    bui = []
    fwi = []
    for index, x in df.iterrows():
        ffmc.append(FFMC(x['air'],x['rhum'],x['uwnd'], x['precip'], FFMCPrev=ffmc_prev))
        dmc.append(DMC(x['air'],x['rhum'],x['precip'], dmc_prev, x['latitude'],x['month']))
        dc.append(DC(x['air'], x['precip'], dc_prev,x['latitude'], x['month']))
        isi.append(ISI(x['uwnd'], ffmc[-1]))
        bui.append(BUI(dmc[-1], dc[-1]))
        fwi.append(FWI(isi[-1], bui[-1]))
        fmc_prev = ffmc[-1]
        dmc_prev = dmc[-1]
        dc_prev = dc[-1]
        
    df['ffmc'] = ffmc
    df['dmc'] = dmc
    df['dc'] = dc
    df['isi'] = isi
    df['bui'] = bui
    df['fwi'] = fwi
    
    df.drop(['lon', 'lat', 'lon_int', 'lat_int'], axis=1, inplace=True, errors='ignore')


def add_osm_features(df):
    with gzip.open(OSM_GEO_DATA, 'rb') as f:
        osm_df = geopandas.read_file(f, crs="epsg:4326")
    POINT_SIZE_X = 0.1
    POINT_SIZE_Y = 0.1
    geo_df = df.reset_index()
    geo_df = geopandas.GeoDataFrame(
        geo_df[['fire_id']],
        geometry=geo_df.apply(lambda x: box(
            x.longitude - POINT_SIZE_X / 2, x.latitude - POINT_SIZE_Y / 2,
            x.longitude + POINT_SIZE_X / 2, x.latitude + POINT_SIZE_Y / 2
        ), axis=1), crs="epsg:4326")

    geo_features = geopandas. \
        sjoin(geo_df, osm_df.drop(['ids', 'names'], axis=1), how='left', op='intersects'). \
        drop(['geometry', 'index_right'], axis=1). \
        groupby('fire_id'). \
        mean().fillna(0)

    for col in geo_features.columns:
        df[col] = geo_features[col]

def prepare_dataset(filename):
    df = pd.read_csv(filename, parse_dates=['date'])
    preprocess(df)
    add_ncep_features(df)
    add_osm_features(df)
    df = additional_features(df)
    return df

In [7]:
import pickle
from sklearn.model_selection import TimeSeriesSplit, StratifiedShuffleSplit


def train_model(df_train):
    last_month = df_train.ym.max()
    train = df_train[df_train.ym <= last_month - VAL_MONTHS]
    val = df_train[df_train.ym > last_month - VAL_MONTHS]
    
    X_train = train.drop(['fire_type', 'ym', 'date'], axis=1)
    Y_train = train.fire_type
    X_val = val.drop(['fire_type', 'ym', 'date'], axis=1)
    Y_val = val.fire_type
    clf = catboost.CatBoostClassifier(loss_function='MultiClass',
                                      verbose=10, random_state=SEED, iterations=ITERATIONS, early_stopping_rounds=100)
    clf.fit(X_train, Y_train, eval_set=(X_val, Y_val), use_best_model=True)
    pred_train = clf.predict_proba(X_train)
    pred_val = clf.predict_proba(X_val)
    train_scores = evaluate(Y_train, pred_train)
    val_scores = evaluate(Y_val, pred_val)
    print("Train scores:")
    for k, v in train_scores.items():
        print("%s\t%f" % (k, v))
    print("Validation scores:")
    for k, v in val_scores.items():
        print("%s\t%f" % (k, v))
    clf.save_model(os.path.join(MODELS_PATH, 'catboost.cbm'))
    

In [8]:
def train_model_bag(df_train, bags, seed):
    last_month = df_train.ym.max()
    train = df_train[df_train.ym <= last_month - VAL_MONTHS]
    val = df_train[df_train.ym > last_month - VAL_MONTHS]

    X_train = train.drop(['fire_type', 'ym', 'date'], axis=1)
    Y_train = train.fire_type
    X_val = val.drop(['fire_type', 'ym', 'date'], axis=1)

    Y_val = val.fire_type
    
    bagged_prediction = np.zeros((X_val.shape[0], 11))
    for idx, n in enumerate(range(0, bags)):
        clf = catboost.CatBoostClassifier(loss_function='MultiClass',
                                          verbose=10, random_state=seed + n, iterations=ITERATIONS, early_stopping_rounds=100)
        clf.fit(X_train, Y_train, eval_set=(X_val, Y_val), use_best_model=True)
        pred_train = clf.predict_proba(X_train)
        pred_val = clf.predict_proba(X_val)
        
        bagged_prediction = bagged_prediction + pred_val

        train_scores = evaluate(Y_train, pred_train)
        val_scores = evaluate(Y_val, pred_val)
        
        
        print(f"Iter bagging: {idx} Seed: {seed + n}")
        print("Train scores:")
        for k, v in train_scores.items():
            print("%s\t%f" % (k, v))
        print("Validation scores:")
        for k, v in val_scores.items():
            print("%s\t%f" % (k, v))
        clf.save_model(os.path.join(MODELS_PATH, f'catboost_{seed+n}.cbm'))
    
    bagged_prediction = bagged_prediction / bags
    val_scores = evaluate(Y_val, bagged_prediction)
    print("Train scores:")
    for k, v in val_scores.items():
        print("%s\t%f" % (k, v))

Пробовал разные виды кластеризации, но зашел очень хорошо именно MiniBatchKMeans. <br>
Лучший результат дал именно MiniBatchKMeans

quantile - модель показывала их ценность 0, но при их наличии, скор рос на 4й знак, поэтому оставил


In [9]:
def additional_features(df, is_train, n_cluster=100):
    if is_train:
        coords = np.vstack((df[['latitude', 'longitude']].values))
    #     sample_ind = np.random.permutation(len(coords))
        kmeans = MiniBatchKMeans(n_clusters=n_cluster, batch_size=10000, random_state=2019).fit(coords) #[sample_ind])
        df.loc[:, 'lt_lg_cluster'] = kmeans.predict(df[['latitude', 'longitude']])
        filename = 'kmeans_model.sav'
        pickle.dump(kmeans, open(filename, 'wb'))
    else:
        filename = 'kmeans_model.sav'
        kmeans = pickle.load(open(filename, 'rb'))
        df.loc[:, 'lt_lg_cluster'] = kmeans.predict(df[['latitude', 'longitude']])
        
    
    df['air_q95'] = df['air'].quantile(.95)
    df['air_q005'] = df['air'].quantile(.05)
    
    return df

In [10]:
%%time

reseed()
if False:
    df_train = prepare_dataset(os.path.join(DATA_PATH, 'wildfires_train.csv'))
    df_train.head()
else:
    df_train = pd.read_csv("df_train.csv", index_col='fire_id')
    df_train = additional_features(df_train, is_train=False, n_cluster=100)

Wall time: 3.59 s


In [12]:
import datetime

df_train['day_of_year'] = df_train['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").timetuple().tm_yday)

In [11]:
def day_length(dayOfYear, lat):
    latInRad = np.deg2rad(lat)
    declinationOfEarth = 23.45*np.sin(np.deg2rad(360.0*(283.0+dayOfYear)/365.0))
    if -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) <= -1.0:
        return 24.0
    elif -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) >= 1.0:
        return 0.0
    else:
        hourAngle = np.rad2deg(np.arccos(-np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth))))
        return 2.0*hourAngle/15.0

In [13]:
df_train['day_len'] = df_train.apply(lambda x: day_length(x['day_of_year'], x['latitude']), axis=1)

In [14]:
df_train.drop("day_of_year", axis=1, inplace=True)

In [15]:
df_train['pickup_long_15'] = df_train['longitude']*np.cos(15* np.pi / 180) - df_train['latitude']*np.sin(15* np.pi/180)
df_train['pickup_long_30'] = df_train['longitude']*np.cos(30* np.pi / 180) - df_train['latitude']*np.sin(30* np.pi/180)
df_train['pickup_long_45'] = df_train['longitude']*np.cos(45* np.pi / 180) - df_train['latitude']*np.sin(45* np.pi/180)
df_train['pickup_long_60'] = df_train['longitude']*np.cos(60* np.pi / 180) - df_train['latitude']*np.sin(60* np.pi/180)
df_train['pickup_long_75'] = df_train['longitude']*np.cos(75* np.pi / 180) - df_train['latitude']*np.sin(75* np.pi/180)

df_train['pickup_lat_15'] = df_train['longitude']*np.sin(15* np.pi / 180) + df_train['latitude']*np.cos(15* np.pi/180)
df_train['pickup_lat_30'] = df_train['longitude']*np.sin(30* np.pi / 180) + df_train['latitude']*np.cos(30* np.pi/180)
df_train['pickup_lat_45'] = df_train['longitude']*np.sin(45* np.pi / 180) + df_train['latitude']*np.cos(45* np.pi/180)
df_train['pickup_lat_60'] = df_train['longitude']*np.sin(60* np.pi / 180) + df_train['latitude']*np.cos(60* np.pi/180)
df_train['pickup_lat_75'] = df_train['longitude']*np.sin(75* np.pi / 180) + df_train['latitude']*np.cos(75* np.pi/180)

In [16]:
df_train.head()

,date,latitude,longitude,fire_type,weekday,year,day,weekends,passed_years,passed_months,seasons,month,ym,air,air_3d,air_7d,air_14d,air_28d,uwnd,uwnd_3d,uwnd_7d,uwnd_14d,uwnd_28d,rhum,rhum_3d,rhum_7d,rhum_14d,rhum_28d,precip,precip_3d,month_air_mean,month_air_std,seasons_uwnd_mean,seasons_uwnd_std,year_rhum_mean,year_rhum_std,year_month_rhum_mean,year_month_rhum_std,month_rhum_min,month_rhum_max,x,y,z,temp_psd,temp_fft_freq,ffmc,dmc,dc,isi,bui,fwi,city,town,village,neighbourhood,hamlet,locality,continent,suburb,isolated_dwelling,allotments,island,region,sea,county,mountain_range,peninsula,quarter,islet,country,state,farm,archipelago,islands,allotments_set,historic,subdistrict,square,wall,дом Малькова,plot,yard,neighbouhood,unknown,wood,school,water,yes,residential,wetland,forest,commercial,apartments,scrub,public,university,stadium,grassland,hospital,reservoir,office,college,kindergarten,clinic,grass,meadow,hotel,peat_cutting,farmland,industrial,construction,education,garages,heath,quarry,village_green,fell,spit,municipality,policlinic,civic,store,recreation_ground,landfill,cemetery,train_station,national_reserve,orchard,farmyard,sand,retail,beach,castle,offices,railway,bay,natural_reserve,lava,station,military,greenfield,cathedral,mud,dormitory,brownfield,service,grandstand,building:part,house,garage,roof,church,goverment,greenhouse_horticulture,basin,depot,lt_lg_cluster,air_q95,air_q005,day_len,pickup_long_15,pickup_long_30,pickup_long_45,pickup_long_60,pickup_long_75,pickup_lat_15,pickup_lat_30,pickup_lat_45,pickup_lat_60,pickup_lat_75
fire_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2012-01-01,42.913440,133.88737,4,6,2012,1,0,7,94,1,1,145,266.20000,264.865,264.76248,264.47858,263.85928,-0.919998,-0.095001,1.525001,2.695716,4.110000,73.0,72.000000,69.937500,69.785710,63.87500,0.000000,0.112435,258.70676,7.605233,0.369371,4.345529,52.16531,15.870315,73.0,1.457738,31.720001,100.0,-0.173843,0.316755,-0.876651,51.115820,0.000000,135.528620,14.884972,62.620002,421.419970,18.673230,148.263718,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61,298.7,266.74,8.887238,118.218453,94.493144,64.328283,29.779556,-6.798599,76.103801,104.107814,125.017052,137.406584,140.432084
1,2012-01-01,43.378616,131.77226,3,6,2012,1,0,7,94,1,1,145,260.33002,259.790,260.30002,260.82570,261.05072,0.979996,1.384994,2.412495,3.311423,3.004281,73.5,73.750000,74.405000,71.622856,67.26500,0.000000,0.113171,258.70676,7.605233,0.369371,4.345529,52.16531,15.870315,73.0,1.457738,31.720001,100.0,0.810731,-0.559043,-0.567677,31.658865,0.002087,133.656796,23.413904,109.183405,445.807774,30.484590,181.618744,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24,298.7,266.74,8.833063,116.055017,92.428817,62.503745,28.319147,-7.795355,76.005696,103.453113,123.850372,135.807433,138.509441
2,2012-01-01,42.634132,130.47911,4,6,2012,1,0,7,94,1,1,145,260.91998,260.000,260.20000,259.97144,260.31073,2.020004,1.800003,2.657501,3.147143,2.516428,70.5,67.875000,67.375000,67.214290,66.44643,0.000000,4.112010,258.70676,7.605233,0.369371,4.345529,52.16531,15.870315,73.0,1.457738,31.720001,100.0,0.022705,

Идеи для вдохновения при генерации фич я черпал из корреляции. <br>

Было 2 основные: Спирмена (т.к. ей не важен тип распределения и можно использовать <br>
и для категориальных признаков)

C mean_target_encoding все сильно сложнее - сами признаки на нем давали жуткое переобучение <br>
поэтому не использовались. Признаки, которые в виде target_mean_encoding давали самую большую корреляцию Спирмена, <br>
вероятнее всего
и содержат какие-то нужные закономерности, которые хорошо коррелируют с целевой переменной. 
Но это лишь гипотеза, интерес еще и в том, что часть этих признаков вели к переобучению (например, дублируя 
информацию - мультиколлинеарность), поэтому каждый из новых признаков нужно было еще проверить.

Можно было обойтись и просто Спирменом, но показалось, что результат чуть хуже.

In [27]:
for col in df_train.columns[:]:
    targetc = KFoldTargetEncoderTrain(col,'fire_type',n_fold=5)
    df_train = targetc.fit_transform(df_train)

Correlation between the new feature, date_Kfold_Target_Enc and, fire_type                    is SpearmanrResult(correlation=0.31955423175305964, pvalue=0.0).
Correlation between the new feature, latitude_Kfold_Target_Enc and, fire_type                    is SpearmanrResult(correlation=0.008334808270277407, pvalue=0.000491308676129847).
Correlation between the new feature, longitude_Kfold_Target_Enc and, fire_type                    is SpearmanrResult(correlation=0.0144748759595385, pvalue=1.4189354249352111e-09).
Correlation between the new feature, fire_type_Kfold_Target_Enc and, fire_type                    is SpearmanrResult(correlation=1.0, pvalue=0.0).
Correlation between the new feature, weekday_Kfold_Target_Enc and, fire_type                    is SpearmanrResult(correlation=0.014709234590239498, pvalue=7.68335653899256e-10).
Correlation between the new feature, year_Kfold_Target_Enc and, fire_type                    is SpearmanrResult(correlation=0.06551795861802721, pvalue=1.2

Одна модель в итоге давала около 0.9293 (на PLB .9308)

В итоге, запустил беггинг (изменяя seed) и получил 0.9314.

т.к. данный ноутбук выжимка из исследования, то тут я не стал ждать пока <br>
модели посчитаются (~примерно 4часа) - чтобы видеть все в логе 

Была идея еще со стекингом, но основная проблема, то что это временные ряды, 
а значит автокорреляция, значит, KFold не подойдет, если только модель не обучалась через 
fold'ы и давала хороший скор (но это не мой случай StratifiedShuffleSplit - давал на данном
наборе очень низкий сокр, хотя до какого-то момента помогал выбивать больше .93). 
Значит, нужно было добавлять шум к признакам или учить на отложенных выборках, но минимум нужно 
было откладывать 1 год. В общем, с этим подходом я немного поиграл и до конца не стал доводить.

In [ ]:
%%time
train_model_bag(df_train, bags=10, seed=SEED)

Validation scores roc_auc_micro 0.929105-0.929741, bagging моделей дал .9314, <br>
к сожалению, скор на валидации беггинга я уже не помню, но что-то около .93

# Выводы
1. Нужно было исследовать наборы признаков под каждую модель отдельно. <br>
Текущий набор хорошо работает с Catboost, но из ряда вон плохо, с LGBM/XGB/RF/ET. <br>
при этом на каких-то наборах эти методы работали даже лучше catboost'a. Потом уже блендить/стекать,
в силу разнородных моделей, думаю, это дало бы увеличение скора.

2. Самый большой проблемой стал мерж данных по координатам + дате <br>
Если для большинства наборов ncep/eslr с этим возни было не так много, то для firms, это 
стало большой проблемой. Координаты сильно расходились.
Для мержа я нашел 2 самых частых метода:
  * geopandas sjoin (на основе rtree)
  * на основе гавер расстояния.
С geopandas ничего сложного, за исключением того, что bonding box был точкой,
соответственно развернуть в некоторую область не получилось.

Гавер расстояние считается очень долго, для набора 170k ~ 2ч, у firms m6 ~ 2.5М записей,
я так и не дождался. 
В итоге, работал топорно, и округлул до 2х знаков, до целого координаты и мержил (precip)
Но над этим пунктом нужно еще поработать.

3. Много хороших признаков в виде показателей и индексов, уже рассчитаны давно зарубежными коллегами,
я использовал fwi и т.д. (эти метрики есть, например, в наборе https://archive.ics.uci.edu/ml/datasets/Forest+Fires)
Есть, например,  EMC, который тоже добавляет к качеству.

df_train['EMC_10'] = 0.03229 + 0.281073* df_train['rhum'] - 0.000578 * df_train['air'] * df_train['rhum']
df_train['EMC_10_50'] = 2.22749 + 0.160107 * df_train['rhum'] - 0.014784 * df_train['rhum']
df_train['EMC_50'] = 21.0606 + 0.005565 * df_train['rhum'] ** 2 - 0.00035 * df_train['rhum'] * df_train['air'] - 0.483199 * df_train['rhum']

Но в свое время когда считал модели, организаторы все тянули с данными. Поэтому большую часть этих индексов я так и не считал.
А после объявления списка данных, уже не было ни желания, ни времени считать (считается-то не 5 минут), поэтому последние несколько дней, особо и не занимался улучшением скора, просто пробовал новые признаки, которые где-то давали улучшение, но прирост был локально маленький, поэтому считать все стал.

4. Еще большой проблемой стало, что организаторы до последнего тянули со списком данных. Эта проблема касалась всех участников,
и на мой взгляд, это было большим промахом организаторов и остается на их совести. Я предупреждал о проблеме, когда нужного набора 
не будет в итоге из-за слишком большого списка наборов и слишком широкой трактовки, то и получили precip нет в итоге, а значит, скор может сместиться (хотя  и не сильно, для зимы осадки с около 0 в целом давали неплохой скор). Нужно было обозначать списки раньше, а не перед окончанием. Думаю, многие согласятся, что из-за этой проблемы, другие данные не было особой мотивации даже смотреть. Честно говоря, учавствовать в соревновании с похожим раскладом больше не хочется

Но в целом все же положительно хочется отметить работу организаторов, другие вопросы решались хорошо, а проблему выше принять к рассмотрению, на будущее.